In [2]:
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving 서울시 주요 공원현황.csv to 서울시 주요 공원현황.csv


In [1]:
import numpy as np
import pandas as pd
import folium
import re

In [3]:
park_df = pd.read_csv(filename, encoding = 'EUC-KR')
park_df.head(1)

,연번,공원명,공원개요,면적,개원일,주요시설,주요식물,안내도,오시는길,이용시참고사항,이미지,지역,공원주소,관리부서,전화번호,X좌표(GRS80TM),Y좌표(GRS80TM),X좌표(WGS84),Y좌표(WGS84),바로가기
0,1,남산도시자연공원,남산공원은 도심에 위치하여 서울시민에게 맑은 공기를 제공하는 자연휴식처이며 산책 꽃...,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,1968.9.10,기반시설 : 광장 45950㎡ 도로 108530㎡ 산책로 6.7㎞ (북측:3.7㎞/...,소나무 단풍 아카시아 상수리나무 등 191종 2881870주,http://parks.seoul.go.kr/upload/seditorMulti/2...,명동역(남산케이블카 와룡묘 서울애니메이션센터 방면) 도보 명동역 3번출구 → 퍼시픽...,남산공원 차량통행 금지안내 2005년 5월 1일부터 남산공원에 일반승용차 택시 통행...,http://parks.seoul.go.kr/file/info/view.do?fId...,중구,서울특별시 중구 삼일대로 231(예장동),서울특별시중부푸른도시사업소,02-3783-5900,198364.107,450395.554,126.990377,37.55014,http://parks.seoul.go.kr/template/sub/namsan.do


## 1. 공원 면적 계산

In [4]:
area = park_df.면적.astype(str).apply(lambda x: x.strip('총 ').split()[0])
area[-5:]

127    109635㎡
128    140022㎡
129    211392㎡
130    157535㎡
131        nan
Name: 면적, dtype: object

In [5]:
area[(area == 'nan') | (area == '휴양')]

6       휴양
109    nan
119    nan
131    nan
Name: 면적, dtype: object

In [6]:
area[(area == 'nan') | (area == '휴양')] = '0㎡'
area[-5:]

127    109635㎡
128    140022㎡
129    211392㎡
130    157535㎡
131         0㎡
Name: 면적, dtype: object

In [8]:
area = area.apply(lambda x: float(re.sub('[㎡m²㎥]', '', x)))

## 2. 공원 면적을 크기에 따라 분류

In [10]:
area_criteria = [-1, 100000, 1000000, 12000000]
labels = ['소형', '중형', '대형']
size_info = [3, 7, 15]
scale = pd.cut(area, area_criteria, labels=labels)
size = pd.cut(area, area_criteria, labels=size_info)
scale[:5]

0    대형
1    소형
2    대형
3    중형
4    대형
Name: 면적, dtype: category
Categories (3, object): ['소형' < '중형' < '대형']

In [11]:
size[:5]

0    15
1     3
2    15
3     7
4    15
Name: 면적, dtype: category
Categories (3, int64): [3 < 7 < 15]

## 3. 데이터프레임 (공원명, 지역, 면적, 위도, 경도, 분류, 크기)

In [12]:
seoul_park_df = park_df[['공원명', '지역', 'X좌표(WGS84)', 'Y좌표(WGS84)']]
seoul_park_df.columns = ['공원명', '지역', '경도', '위도']
seoul_park_df['면적'] = area
seoul_park_df['분류'] = scale
seoul_park_df['크기'] = size
seoul_park_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

,공원명,지역,경도,위도,면적,분류,크기
0,남산도시자연공원,중구,126.990377,37.550140,2896887.0,대형,15
1,길동생태공원,강동구,127.154779,37.540394,80683.0,소형,3
2,서울대공원,과천시,127.019847,37.426449,9132690.0,대형,15
3,서울숲,성동구,127.041798,37.543072,480994.0,중형,7
4,월드컵공원,마포구,126.878907,37.571805,2284085.0,대형,15


In [13]:
seoul_park_df.isnull().sum()

공원명    0
지역     1
경도     1
위도     1
면적     0
분류     0
크기     0
dtype: int64

In [14]:
seoul_park_df[seoul_park_df.경도.isna()]

,공원명,지역,경도,위도,면적,분류,크기
131,서울로7017,중구,NaN,NaN,0.0,소형,3


In [15]:
seoul_park_df = seoul_park_df.dropna()

In [16]:
seoul_park_df.to_csv('SeoulPark.csv', index=False)

## 4. 서울시 지도에 공원 시각화

In [25]:
center = [37.3306890, 126.5930664] # Centeral Location of Seoul
map = folium.Map(location = center, zoom_start = 10)
for i in seoul_park_df.index:
    folium.CircleMarker(
        [seoul_park_df.위도[i], seoul_park_df.경도[i]],
        radius = int(seoul_park_df.크기[i]),
        tooltip = f"{seoul_park_df.공원명[i]}({seoul_park_df.면적[i]:}㎡)",
        color = 'green', fill_color = 'green'
).add_to(map)
title = '<h3 align = "center" style = "font-size:20px"><b>Seoul Park<b></h3>'
map

In [26]:
map.save('SeoulPark.html')